## Chapter 5.文件与IO
>使用的模块：
1. array
2. os
 - os.path
3. io
 - io.StringIO
 - io.BytesIO
4. gzip
5. bz2
6. tempfile
 - tempfile.TemporaryFile
 - tempfile.NamedTemporaryFile
7. pickle
 - pickle.dump
 - pickle.load
8. sys
9. glob
10. fnmatch

#### 1. 读写文本数据
使用带有 rt 模式的 open() 函数读取文本文件。  
使用带有 wt 模式的 open() 函数写文件， 如果之前文件内容存在则清除并覆盖掉。  
如果是在已存在文件中添加内容，使用模式为 at 的 open() 函数。

In [ ]:
# Iterate over the lines of the file
with open('somefile.txt', 'rt') as f:
    for line in f:
        # process line
        ...
        
# Write chunks of text data
with open('somefile.txt', 'wt') as f:
    f.write(text1)
    f.write(text2)

#如果你已经知道你要读写的文本是其他编码方式， 那么可以通过传递一个可选的 encoding 参数给open()函数
with open('somefile.txt', 'rt', encoding='latin-1') as f:
    ...

    Character Meaning
--------- ---------------------------------------------------------------
    'r'       open for reading (default)  
    'w'       open for writing, truncating the file first  
    'x'       create a new file and open it for writing  
    'a'       open for writing, appending to the end of the file if it exists  
    'b'       binary mode  
    't'       text mode (default)  
    '+'       open a disk file for updating (reading and writing)  
    'U'       universal newline mode (deprecated)

几个常见的编码是ascii, latin-1, utf-8和utf-16。   
在web应用程序中通常都使用的是UTF-8。   
ascii对应从U+0000到U+007F范围内的7位字符。   
latin-1是字节0-255到U+0000至U+00FF范围内Unicode字符的直接映射。 当读取一个未知编码的文本时使用latin-1编码永远不会产生解码错误。 使用latin-1编码读取一个文件的时候也许不能产生完全正确的文本解码数据， 但是它也能从中提取出足够多的有用数据。同时，如果你之后将数据回写回去，原先的数据还是会保留的。

**关于换行符的识别问题**：
在Unix和Windows中是不一样的(分别是 \n 和 \r\n )。 默认情况下，Python会以统一模式处理换行符。 这种模式下，在读取文本的时候，Python可以识别所有的普通换行符并将其转换为单个 \n 字符。 类似的，在输出时会将换行符 \n 转换为系统默认的换行符。 如果你不希望这种默认的处理方式，可以给 open() 函数传入参数 newline='' ，就像下面这样：

In [ ]:
# Read with disabled newline translation
with open('somefile.txt', 'rt', newline='') as f:
    ...

**使用其他分隔符或行终止符打印**

In [1]:
print('ACME', 50, 91.5)
print('ACME', 50, 91.5, sep=',')
print('ACME', 50, 91.5, sep=',', end='!!\n')

row = ('ACME', 50, 91.5)
print(*row, sep=',')

ACME 50 91.5
ACME,50,91.5
ACME,50,91.5!!
ACME,50,91.5


#### 2.读写字节数据
使用模式为 rb 或 wb 的 open() 函数来读取或写入二进制数据。

In [ ]:
# Read the entire file as a single byte string
with open('somefile.bin', 'rb') as f:
    data = f.read()

# Write binary data to a file
with open('somefile.bin', 'wb') as f:
    f.write(b'Hello World')
#在写入的时候，必须保证参数是以字节形式对外暴露数据的对象(比如字节字符串，字节数组对象等)

如果你想从二进制模式的文件中读取或写入文本数据，必须确保要进行解码和编码操作。

In [ ]:
with open('somefile.bin', 'rb') as f:
    data = f.read(16)
    text = data.decode('utf-8')

with open('somefile.bin', 'wb') as f:
    text = 'Hello World'
    f.write(text.encode('utf-8'))

二进制I/O还有一个鲜为人知的特性就是数组和C结构体类型能直接被写入，而不需要中间转换为自己对象，这个适用于任何实现了被称之为”缓冲接口”的对象，这种对象会直接暴露其底层的内存缓冲区给能处理它的操作。 二进制数据的写入就是这类操作之一。

In [ ]:
import array
nums = array.array('i', [1, 2, 3, 4])
with open('data.bin','wb') as f:
    f.write(nums)

在写文件时通常会遇到的一个问题的完美解决方案(不小心覆盖一个已存在的文件)。 一个替代方案是先测试这个文件是否存在，像下面这样：

In [3]:
import os
if not os.path.exists('somefile'):
    with open('somefile', 'wt') as f:
        f.write('Hello\n')
else:
    print('File already exists!')

File already exists!


字符串的I/O操作：  
使用 io.StringIO() 和 io.BytesIO() 类来创建类文件对象操作字符串数据。

In [5]:
import io
s = io.StringIO()
s.write('Hello World\n')
print('This is a test', file=s)

# Get all of the data written so far
print(s.getvalue())

# Wrap a file interface around an existing string
s = io.StringIO('Hello\nWorld\n')
print(s.read(4))
print(s.read())

Hello World
This is a test

Hell
o
World



#### 3. 读写压缩文件
gzip 和 bz2 模块可以很容易的处理这些文件。

In [ ]:
# gzip compression
import gzip
with gzip.open('somefile.gz', 'rt') as f:
    text = f.read()

# bz2 compression
import bz2
with bz2.open('somefile.bz2', 'rt') as f:
    text = f.read()
    
# gzip compression
import gzip
with gzip.open('somefile.gz', 'wt') as f:
    f.write(text)

# bz2 compression
import bz2
with bz2.open('somefile.bz2', 'wt') as f:
    f.write(text)
    
#使用 compresslevel 这个可选的关键字参数来指定一个压缩级别,默认的等级是9，也是最高的压缩等级。等级越低性能越好，但是数据压缩程度也越低。
with gzip.open('somefile.gz', 'wt', compresslevel=5) as f:
    f.write(text)

#### 4.文件路径名的操作
使用 os.path 模块中的函数来操作路径名:  
os.path 模块知道Unix和Windows系统之间的差异并且能够可靠地处理类似 Data/data.csv 和 Data\data.csv 这样的文件名。

In [6]:
import os
path = '/Users/beazley/Data/data.csv'

# Get the last component of the path
print(os.path.basename(path))

# Get the directory name
print(os.path.dirname(path))

# Join path components together
print(os.path.join('tmp', 'data', os.path.basename(path)))

# Expand the user's home directory
path = '~/Data/data.csv'
print(os.path.expanduser(path))

# Split the file extension
print(os.path.splitext(path))

data.csv
/Users/beazley/Data
tmp\data\data.csv
C:\Users\Hughie/Data/data.csv
('~/Data/data', '.csv')


**测试文件是否存在**

In [ ]:
import os
print(os.path.exists('/etc/passwd'))
print(os.path.exists('/tmp/spam'))

# Is a regular file
os.path.isfile('/etc/passwd')

# Is a directory
os.path.isdir('/etc/passwd')

# Is a symbolic link
os.path.islink('/usr/local/bin/python3')

# Get the file linked to
os.path.realpath('/usr/local/bin/python3')

#获取元数据(比如文件大小或者是修改日期)
print(os.path.getsize('/etc/passwd'))
print(os.path.getmtime('/etc/passwd'))
import time
print(time.ctime(os.path.getmtime('/etc/passwd')))

**获取文件夹中的文件列表**  
os.listdir(),结果会返回目录中所有文件列表，包括所有文件，子目录，符号链接等等

In [ ]:
import os.path

# Get all regular files
names = [name for name in os.listdir('somedir') if os.path.isfile(os.path.join('somedir', name))]

# Get all dirs
dirnames = [name for name in os.listdir('somedir') if os.path.isdir(os.path.join('somedir', name))]

 如果你还想获取其他的元信息，比如文件大小，修改时间等等， 你或许还需要使用到 os.path 模块中的函数或着 os.stat() 函数来收集数据。

In [ ]:
# Example of getting a directory listing

import os
import os.path
import glob

pyfiles = glob.glob('*.py')

# Get file sizes and modification dates
name_sz_date = [(name, os.path.getsize(name), os.path.getmtime(name)) for name in pyfiles]
for name, size, mtime in name_sz_date:
    print(name, size, mtime)

# Alternative: Get file metadata
file_metadata = [(name, os.stat(name)) for name in pyfiles]
for name, meta in file_metadata:
    print(name, meta.st_size, meta.st_mtime)

**忽略文件名编码**

In [ ]:
# Wrte a file using a unicode filename
with open('jalape\xf1o.txt', 'w') as f:
    f.write('Spicy!')

# Directory listing (decoded)
import os
os.listdir('.')

# Directory listing (raw)
os.listdir(b'.') # Note: byte string

# Open file with raw filename
with open(b'jalapen\xcc\x83o.txt') as f:
    print(f.read())

#### 5.  打印不合法的文件名
你的程序获取了一个目录中的文件名列表，但是当它试着去打印文件名的时候程序崩溃， 出现了 UnicodeEncodeError 异常和一条奇怪的消息—— surrogates not allowed 。

In [ ]:
def bad_filename(filename):
    return repr(filename)[1:-1]

def bad_filename(filename):
    temp = filename.encode(sys.getfilesystemencoding(), errors='surrogateescape')
    return temp.decode('latin-1')

try:
    print(filename)
except UnicodeEncodeError:
    print(bad_filename(filename))

默认情况下，Python假定所有文件名都已经根据 sys.getfilesystemencoding() 的值编码过了。 但是，有一些文件系统并没有强制要求这样做，因此允许创建文件名没有正确编码的文件。 这种情况不太常见，但是总会有些用户冒险这样做或者是无意之中这样做了( 可能是在一个有缺陷的代码中给 open() 函数传递了一个不合规范的文件名)。

**增加或改变已打开文件的编码**  
如果你想修改一个已经打开的文本模式的文件的编码方式，可以先使用 detach() 方法移除掉已存在的文本编码层， 并使用新的编码方式代替。

In [ ]:
import sys
sys.stdout.encoding
sys.stdout = io.TextIOWrapper(sys.stdout.detach(), encoding='latin-1')
sys.stdout.encoding

#### 6. 创建临时文件和文件夹
tempfile 模块中有很多的函数可以完成这任务:

In [ ]:
from tempfile import TemporaryFile

with TemporaryFile('w+t') as f:
    # Read/write to the file
    f.write('Hello World\n')
    f.write('Testing\n')

    # Seek back to beginning and read the data
    f.seek(0)
    data = f.read()

# Temporary file is destroyed

#TemporaryFile() 另外还支持跟内置的 open() 函数一样的参数
with TemporaryFile('w+t', encoding='utf-8', errors='ignore') as f:
    ...

在大多数Unix系统上，通过 TemporaryFile() 创建的文件都是匿名的，甚至连目录都没有。 如果你想打破这个限制，可以使用 NamedTemporaryFile() 来代替。

In [ ]:
from tempfile import NamedTemporaryFile

with NamedTemporaryFile('w+t') as f:
    print('filename is:', f.name)
    ...

# File automatically destroyed

#和 TemporaryFile() 一样，结果文件关闭时会被自动删除掉。 如果你不想这么做，可以传递一个关键字参数 delete=False 即可。
with NamedTemporaryFile('w+t', delete=False) as f:
    print('filename is:', f.name)
    ...

为了创建一个临时目录，可以使用 tempfile.TemporaryDirectory():

In [ ]:
from tempfile import TemporaryDirectory

with TemporaryDirectory() as dirname:
    print('dirname is:', dirname)
    # Use the directory
    ...
# Directory and all contents destroyed

#### 7. 序列化Python对象  
你需要将一个Python对象序列化为一个字节流，以便将它保存到一个文件、存储到数据库或者通过网络传输它。

In [ ]:
import pickle

data = ... # Some Python object
f = open('somefile', 'wb')
pickle.dump(data, f) #将一个对象转储为一个字符串

# Restore from a file
f = open('somefile', 'rb')
data = pickle.load(f)

# Restore from a string
data = pickle.loads(s)

如果要处理多个对象，你可以这样做：

In [9]:
import pickle
f = open('somedata', 'wb')
pickle.dump([1, 2, 3, 4], f)
pickle.dump('hello', f)
pickle.dump({'Apple', 'Pear', 'Banana'}, f)
f.close()

f = open('somedata', 'rb')
print(pickle.load(f))
print(pickle.load(f))
print(pickle.load(f))

[1, 2, 3, 4]
hello
{'Banana', 'Pear', 'Apple'}


千万不要对不信任的数据使用pickle.load()。  
pickle在加载时有一个副作用就是它会自动加载相应模块并构造实例对象。  
但是某个坏人如果知道pickle的工作原理，他就可以创建一个恶意的数据导致Python执行随意指定的系统命令。  
因此，一定要保证pickle只在相互之间可以认证对方的解析器的内部使用。

pickle 对于大型的数据结构比如使用 array 或 numpy 模块创建的二进制数组效率并不是一个高效的编码方式。 如果你需要移动大量的数组数据，你最好是先在一个文件中将其保存为数组数据块或使用更高级的标准编码方式如HDF5 (需要第三方库的支持)。